In [ ]:
basename = 'Tomato10-2path-LAB-m4-'
monitor='val_accuracy'
epochs=150
batch_size=32
input_shape=(224, 224, 3) # please resize it to (224,224,3) if you have enough RAM
Verbose=True

In [ ]:
import numpy as np

import sys
print("Python version:",sys.version)

import skimage
print('skimage version:', skimage.__version__)

import tensorflow as tf
print('Tensorflow:', tf.__version__)

import multiprocessing

import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#apt-get install git python3-opencv
import os

if not os.path.isdir('k'):
  !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull

!cd k && pip install .

!rm -rf k

import cai
import cai.datasets
import cai.models

In [ ]:
data_dir = "/kaggle/input/tomato-leave-diseases-dataset"
label_of_classes = os.listdir(data_dir)
print("Class lables:", label_of_classes)

number_of_classes = len(label_of_classes)
print("Number of classes:", number_of_classes)

train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(
    seed=7,
    root_dir=data_dir,
    lab=True,
    verbose=Verbose,
    bipolar=False,
    base_model_name='tomato_leaf',
    training_size=0.6,
    validation_size=0.2,
    test_size=0.2,
    target_size=(input_shape[0],input_shape[1]), 
    has_training=True,
    has_validation=True,
    has_testing=True, 
    smart_resize=True
)

print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

# Fiting Model

In [ ]:
for l_ratio in [0.0,0.25,0.50,0.75,1.00]:
    basefilename = basename + str(l_ratio)

    print('Running: '+basefilename)
    
    model = cai.models.compiled_two_path_inception_v3(
        input_shape=input_shape,
        classes=number_of_classes, 
        two_paths_first_block=True,
        two_paths_second_block=False,
        l_ratio=l_ratio,
        ab_ratio=(1-l_ratio),
        max_mix_idx=4, 
        model_name='two_path_inception_v3'
    )    
       
    best_result_file_name = basefilename+'-best_result.hdf5'
    
    save_best = tf.keras.callbacks.ModelCheckpoint(
        filepath=best_result_file_name, 
        monitor=monitor, 
        verbose=1, 
        save_best_only=True, 
        save_weights_only=False, 
        mode='max', 
        save_freq='epoch'
    )
    
    history = model.fit(
        train_x,
        train_y,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(val_x,val_y),
        callbacks=[save_best],
        class_weight=classweight,
        workers=multiprocessing.cpu_count()
    )
    
#     all_historeis.append(history)
    
    print('Testing Last Model: '+basefilename)
    
    evaluated = model.evaluate(test_x,test_y)
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
        print(name,metric)

#     print('Best Model Results: '+basefilename)
    
#     model = tf.keras.models.load_model(
#         best_result_file_name,
#         custom_objects={'CopyChannels': cai.layers.CopyChannels}
#     )
    
#     evaluated = model.evaluate(test_x,test_y)
    
#     cai.models.save_model(model, basefilename)
    
#     for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
#         print(name,metric)
        
    print('Finished: '+basefilename)

# Test Evaluation

In [ ]:
for l_ratio in [0.0,0.25,0.5,0.75,1.0]:
    basefilename = basename + str(l_ratio)
   
    best_result_file_name = basefilename+'-best_result.hdf5'
    
    print('Best Model Results: '+basefilename)
    
    model = tf.keras.models.load_model(
        best_result_file_name,
        custom_objects={'CopyChannels': cai.layers.CopyChannels}
    )
    
    evaluated = model.evaluate(test_x,test_y)
    
    cai.models.save_model(model, basefilename)
    
    for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
        print(name,metric)
        
    print('Finished: '+basefilename)

# F1 Calculation

In [ ]:
for l_ratio in [0.0,0.25,0.5,0.75,1.0]:
    basefilename = basename + str(l_ratio)
    
    best_result_file_name = basefilename+'-best_result.hdf5'
    
    print('Best Model Results: '+basefilename)
    
    model = tf.keras.models.load_model(
        best_result_file_name,
        custom_objects={'CopyChannels': cai.layers.CopyChannels}
    )
    
    pred_y = model.predict(test_x)
    #         print("Predicted Shape:", pred_y.shape)
    pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
    test_classes_y = np.array(list(np.argmax(test_y, axis=1)))
    #         print("Pred classes shape:",pred_classes_y.shape)
    #         print("Test classes shape:",test_classes_y.shape)
    report = classification_report(test_classes_y, pred_classes_y, digits=4)
    print(report)

# Confusion Matrix

In [ ]:
for l_ratio in [0.0,0.25,0.5,0.75,1.0]:
    basefilename = basename + str(l_ratio)
     
    best_result_file_name = basefilename+'-best_result.hdf5'          

    print('Best Model Results: '+basefilename)          
    model = tf.keras.models.load_model(
        best_result_file_name,
        custom_objects={'CopyChannels': cai.layers.CopyChannels}
    )

    # Make predictions
    y_pred_prob = model.predict(test_x)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Create the confusion matrix
    cm = confusion_matrix(np.argmax(test_y, axis=1), y_pred)

    # Visualize the confusion matrix using seaborn
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=False, fmt='d', cmap='Blues', cbar=False,
                xticklabels=label_of_classes, yticklabels=label_of_classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    figfilename = basefilename+'-CF.png'
    plt.savefig(figfilename, bbox_inches='tight')

    plt.show()

# Save all histories as a single pickle file (optional)
with open('all_training_histories.pkl', 'wb') as file:
    pickle.dump(all_historeis, file)

In [ ]:
# HISTORY
def plot_history(history, filename):
    
    plt.figure(figsize=(10, 6)) 
    

    # Plot training & validation loss values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper right')
#     plt.ylim(ymin=0, ymax=1.5)

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')    
    plt.legend(['Train', 'Validation'], loc='lower right')
    
    plt.savefig(filename)

    plt.tight_layout()
    plt.show()